In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import time
from datetime import datetime, timedelta
from topaz import TopazAPI
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
import requests
import os
import concurrent.futures
import math
api_key = '51066aa1-85af-4f6f-aaa5-08b0f3133af5' #Insert your API key 
topaz_api = TopazAPI(api_key)
import mapping


In [2]:
runs_df = pd.concat([pd.read_feather('server_data/'+x) for x in os.listdir('server_data/')])

In [3]:
runs_df.isna().sum()

trackCode              0
track                  0
distance               0
raceId                 0
meetingDate            0
                  ...   
position_1        652442
splitMargin_1    1323903
time_2           1854912
position_2       1854912
splitMargin_2    2149796
Length: 65, dtype: int64

In [4]:
runs_df.raceId.nunique()

247743

In [5]:
runs_df = runs_df.dropna(subset=['place'], how='all')    
runs_df.drop_duplicates(inplace=True)

In [6]:
runs_df.raceId.nunique()

245463

In [7]:
runs_df.shape

(1770293, 65)

In [8]:
runs_df.columns

Index(['trackCode', 'track', 'distance', 'raceId', 'meetingDate',
       'raceTypeCode', 'raceType', 'runId', 'dogId', 'dogName', 'weightInKg',
       'incomingGrade', 'outgoingGrade', 'gradedTo', 'rating', 'raceNumber',
       'boxNumber', 'boxDrawnOrder', 'rugNumber', 'startPrice', 'place',
       'unplaced', 'unplacedCode', 'scratched', 'prizeMoney', 'resultTime',
       'resultMargin', 'resultMarginLengths', 'startPaceCode', 'jumpCode',
       'runLineCode', 'firstSecond', 'colourCode', 'sex', 'comment', 'ownerId',
       'trainerId', 'ownerName', 'ownerState', 'trainerName', 'trainerSuburb',
       'trainerState', 'trainerDistrict', 'trainerPostCode', 'isQuad',
       'isBestBet', 'damId', 'damName', 'sireId', 'sireName', 'dateWhelped',
       'totalFormCount', 'last5', 'isLateScratching', 'bestTime',
       'bestFinishTrackAndDistance', 'pir', 'careerPrizeMoney', 'averageSpeed',
       'time_1', 'position_1', 'splitMargin_1', 'time_2', 'position_2',
       'splitMargin_2'],
     

In [9]:
runs_df.track

0        Warrnambool
1        Warrnambool
3        Warrnambool
4        Warrnambool
5        Warrnambool
            ...     
83426    Healesville
83427    Healesville
83428    Healesville
83429    Healesville
83430    Healesville
Name: track, Length: 1770293, dtype: object

In [10]:
runs_df['date'] = pd.to_datetime(runs_df['meetingDate']).dt.date

In [11]:
runs_df['year-month'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('M')
runs_df['year'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('Y')

C:\Users\Nick\AppData\Local\Temp\ipykernel_15048\74229333.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  runs_df['year-month'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('M')
C:\Users\Nick\AppData\Local\Temp\ipykernel_15048\74229333.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  runs_df['year'] = pd.to_datetime(runs_df['meetingDate']).dt.to_period('Y')


In [12]:
runs_df.date.min()

datetime.date(2015, 1, 1)

In [13]:
runs_df.groupby('year')['raceId'].unique().apply(len).value_counts()

raceId
26475    1
27171    1
28027    1
27008    1
26213    1
26166    1
27938    1
24009    1
23342    1
9114     1
Name: count, dtype: int64

In [14]:
TrackDict = {
    'Auckland (NZ)':'Manukau',
    'Christchurch (NZ)':'Addington',
    'Dport @ HOB':'Hobart',
    'Dport @ LCN':'Launceston',
    'Meadows (MEP)':'The Meadows',
    'Otago (NZ)':'Forbury Park',
    'Palmerston Nth (NZ)':'Manawatu',
    'Sandown (SAP)':'Sandown Park',
    'Southland (NZ)':'Ascot Park',
    'Tokoroa (NZ)':'Tokoroa',
    'Waikato (NZ)':'Cambridge',
    'Wanganui (NZ)':'Hatrick',
    'Taranaki (NZ)':'Taranaki',
    'Ashburton (NZ)':'Ashburton',
    'Richmond (RIS)':'Richmond Straight',
    'Murray Bridge (MBR)':'Murray Bridge',
    'Murray Bridge (MBS)':'Murray Bridge Straight'
}

In [15]:
TopazData = runs_df.copy()
TopazData['track'] = TopazData['track'].replace(TrackDict)
TopazData['meetingDate'] = pd.to_datetime(TopazData['meetingDate'])
TopazData['dateWhelped'] = pd.to_datetime(TopazData['dateWhelped'])

TopazData['dogName']=TopazData['dogName'].str.replace("'","")
TopazData['sireName']=TopazData['sireName'].str.replace("'","")
TopazData['damName']=TopazData['damName'].str.replace("'","")

In [16]:
TopazData.raceId.nunique()

245463

In [17]:
state_map = {x['trackName']:x['State'] for x in mapping.trackCodes}

In [18]:
TopazData['state'] = TopazData['track'].map(state_map)

In [19]:
TopazData[['state', 'track']]

,state,track
0,VIC,Warrnambool
1,VIC,Warrnambool
3,VIC,Warrnambool
4,VIC,Warrnambool
5,VIC,Warrnambool
...,...,...
83426,VIC,Healesville
83427,VIC,Healesville
83428,VIC,Healesville
83429,VIC,Healesville


In [20]:
TopazData['last5'] = TopazData['last5'].astype(str)
scaler = MinMaxScaler()
# Function to extract numbers from the 'last5' column
def extract_numbers(row):
    try:
        numbers = list(map(int, row.split('-')))
        # If there are fewer than 5 numbers, pad with zeros
        numbers += [10] * (5 - len(numbers))
        return numbers
    except ValueError:
        # Handle the case where the string cannot be split into integers
        return [10, 10, 10, 10, 10]

# Apply the function to create new columns for each position
TopazData[['positionLastRace', 'positionSecondLastRace', 'positionThirdLastRace',
    'positionFourthLastRace', 'positionFifthLastRace']] = TopazData['last5'].apply(extract_numbers).apply(pd.Series)


#Let's utilise our position columns to generate some win percentages
TopazData['lastFiveWinPercentage'] = ((TopazData[['positionLastRace', 'positionSecondLastRace', 'positionThirdLastRace','positionFourthLastRace', 'positionFifthLastRace']] == 1).sum(axis=1)) / ((TopazData[['positionLastRace', 'positionSecondLastRace', 'positionThirdLastRace','positionFourthLastRace', 'positionFifthLastRace']] != 10).sum(axis=1))
TopazData['lastFiveWinPercentage'].fillna(0,inplace=True)
TopazData['lastFivePlacePercentage'] = ((TopazData[['positionLastRace', 'positionSecondLastRace', 'positionThirdLastRace','positionFourthLastRace', 'positionFifthLastRace']] <= 3).sum(axis=1)) / ((TopazData[['positionLastRace', 'positionSecondLastRace', 'positionThirdLastRace','positionFourthLastRace', 'positionFifthLastRace']] != 10).sum(axis=1))
TopazData['lastFivePlacePercentage'].fillna(0,inplace=True)
TopazData.replace([np.inf, -np.inf], 0, inplace=True)

# Convert the 'pir' column to string
TopazData['pir'] = TopazData['pir'].fillna(0)
TopazData['pir'] = TopazData['pir'].astype(int).astype(str)

# Extract the second last letter and create a new column '2ndLastPIR'
TopazData['2ndLastPIR'] = TopazData['pir'].apply(lambda x: x[-2] if len(x) >= 2 else None)
TopazData['2ndLastPIR'].fillna(TopazData['place'],inplace=True)
TopazData['2ndLastPIR'] = TopazData['2ndLastPIR'].astype(int)

# Create a feature that calculates places gained/conceded in the home straight
TopazData['finishingPlaceMovement'] = TopazData['2ndLastPIR'] - TopazData['place']

TopazData['weightInKgScaled'] = TopazData.groupby('raceId')['weightInKg'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten() if x.nunique() > 1 else 0)

#Scale values as required
TopazData['prizemoneyLog'] = np.log10(TopazData['prizeMoney'] + 1)
TopazData['placeLog'] = np.log10(TopazData['place'] + 1)
TopazData['marginLog'] = np.log10(TopazData['resultMargin'] + 1)

# Calculate median winner time per track/distance
win_results = TopazData[TopazData['place'] == 1]

grouped_data = win_results.groupby(['track', 'distance', 'meetingDate'])['resultTime'].median().reset_index()

median_win_time = pd.DataFrame(grouped_data.groupby(['track', 'distance']).apply(lambda x: x.sort_values('meetingDate').set_index('meetingDate')['resultTime'].shift(1).rolling('365D', min_periods=1).median())).reset_index()
median_win_time.rename(columns={"resultTime": "runTimeMedian"},inplace=True)

median_win_time['speedIndex'] = (median_win_time['runTimeMedian'] / median_win_time['distance'])
median_win_time['speedIndex'] = MinMaxScaler().fit_transform(median_win_time[['speedIndex']])

# Merge with median winner time
TopazData = TopazData.merge(median_win_time, how='left', on=['track', 'distance','meetingDate'])

# Normalise time comparison
TopazData['runTimeNorm'] = (TopazData['runTimeMedian'] / TopazData['resultTime']).clip(0.9, 1.1)
TopazData['runTimeNorm'] = MinMaxScaler().fit_transform(TopazData[['runTimeNorm']])

# Sort the DataFrame by 'RaceId' and 'Box'
TopazData = TopazData.sort_values(by=['raceId', 'boxNumber'])

# Check if there is an entry equal to boxNumber + 1
TopazData['hasEntryBoxNumberPlus1'] = (TopazData.groupby('raceId')['boxNumber'].shift(1) == TopazData['boxNumber'] + 1) | (TopazData['boxNumber'] == 8)
TopazData['hasEntryBoxNumberMinus1'] = (TopazData.groupby('raceId')['boxNumber'].shift(-1) == TopazData['boxNumber'] - 1)
# Convert boolean values to 1
TopazData['hasEntryBoxNumberPlus1'] = TopazData['hasEntryBoxNumberPlus1'].astype(int)
TopazData['hasEntryBoxNumberMinus1'] = TopazData['hasEntryBoxNumberMinus1'].astype(int)
# Display the resulting DataFrame which shows adjacent Vacant Boxes
# Box 1 is treated as having a vacant box to the left always as we are looking how much space the dog has to move.
TopazData['adjacentVacantBoxes'] = 2 - TopazData['hasEntryBoxNumberPlus1'] - TopazData['hasEntryBoxNumberMinus1']
# Calculate 'hasAtLeast1VacantBox'
TopazData['hasAtLeast1VacantBox'] = (TopazData['adjacentVacantBoxes'] > 0).astype(int)

TopazData['win'] = TopazData['place'].apply(lambda x: 1 if x == 1 else 0)

grouped_data = TopazData.groupby(['track', 'distance', 'boxNumber', 'hasAtLeast1VacantBox', 'meetingDate'])['win'].mean().reset_index()
grouped_data.set_index('meetingDate', inplace=True)

# Apply rolling mean calculation to the aggregated data
box_win_percent = grouped_data.groupby(['track', 'distance', 'boxNumber', 'hasAtLeast1VacantBox']).apply(lambda x: x.sort_values('meetingDate')['win'].shift(1).rolling('365D', min_periods=1).mean()).reset_index()

# Reset index and rename columns
box_win_percent.columns = ['track', 'distance', 'boxNumber', 'hasAtLeast1VacantBox', 'meetingDate', 'rolling_box_win_percentage']

# Add to dog results dataframe
TopazData = TopazData.merge(box_win_percent, on=['track', 'distance', 'meetingDate','boxNumber','hasAtLeast1VacantBox'], how='left')

# resultMargin has the same value for 1st and 2nd placed dogs, but should be 0 for the 1st placed dog.
TopazData.loc[TopazData['place'] == 1, ['resultMargin']] = 0

TopazData['dogAge'] = (TopazData['meetingDate'] - TopazData['dateWhelped']).dt.days
scaler = MinMaxScaler()
TopazData['dogAgeScaled'] = TopazData.groupby('raceId')['dogAge'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())

In [53]:
TopazData.shape

(1346273, 93)

In [21]:
import itertools

dataset = TopazData.copy()
print(f"dataset shape: {dataset.shape}")
dataset['meetingDate'] = pd.to_datetime(dataset['meetingDate'])

# Calculate values for dog, trainer, dam and sire
subsets = ['dog', 'trainer', 'dam', 'sire']

# Use rolling window of 28, 91 and 365 days
rolling_windows = ['28D','91D', '365D']

# Features to use for rolling windows calculation
features = ['runTimeNorm', 'placeLog', 'prizemoneyLog', 'marginLog','finishingPlaceMovement']

# Aggregation functions to apply
aggregates = ['min', 'max', 'mean', 'median', 'std']

# Keep track of generated feature names
feature_cols = []

for i in subsets:
    # Generate rolling window features
    idnumber = i + 'Id'

    subset_dataframe = dataset[['meetingDate',idnumber] + features]
    average_df = pd.DataFrame()

    for feature in features:
        # Group by 'damId' and 'meetingDate' and calculate the average of the current feature
        feature_average_df = subset_dataframe.groupby([idnumber, 'meetingDate'])[feature].mean().reset_index()
        # Rename the feature column to indicate it's the average of that feature
        feature_average_df.rename(columns={feature: f'{feature}{i}DayAverage'}, inplace=True)

        # If average_df is empty, assign the feature_average_df to it
        if average_df.empty:
            average_df = feature_average_df
        else:
            # Otherwise, merge feature_average_df with average_df
            average_df = pd.merge(average_df, feature_average_df, on=[idnumber, 'meetingDate'],how='left')

        # Assuming df is your DataFrame
    column_names = average_df.columns.tolist()
    # Columns to exclude
    columns_to_exclude = [idnumber,'meetingDate']
    # Exclude specified columns from the list
    column_names_filtered = [col for col in column_names if col not in columns_to_exclude]

    average_df.drop_duplicates(inplace=True)
    average_df['meetingDate'] = pd.to_datetime(average_df['meetingDate'])
    average_df = average_df.set_index([idnumber, 'meetingDate']).sort_index() 

    for rolling_window in rolling_windows:
        print(f"dataset shape: {dataset.shape}")
        print(f'Processing {i} rolling window {rolling_window} days')

        rolling_result = (
            average_df
            .reset_index(level=0)
            .groupby(idnumber)[column_names_filtered]
            .rolling(rolling_window)  # Use timedelta for rolling window
            .agg(aggregates)
            .groupby(level=0)
            .shift(1)
        )

        # Generate list of rolling window feature names (eg: RunTime_norm_min_365D)
        agg_features_cols = [f'{i}_{f}_{a}_{rolling_window}' for f, a in itertools.product(features, aggregates)]
        # Add features to dataset
        average_df[agg_features_cols] = rolling_result
        # Keep track of generated feature names
        feature_cols.extend(agg_features_cols)
        average_df.fillna(0, inplace=True)

    average_df.reset_index(inplace=True)
    dataset = pd.merge(dataset,average_df,on=[idnumber, 'meetingDate'],how='left')

# Only keep data 12 months after the start date of your dataset since we've used a 365D rolling timeframe for some features
feature_cols = np.unique(feature_cols).tolist()
# dataset = dataset[dataset['meetingDate'] >= '2021-01-01']
dataset = dataset[[
                'meetingDate',
                'state',
                'track',
                'distance',
                'raceId',
                'raceTypeCode',
                'raceNumber',
                'boxNumber',
                'rugNumber',
                'runId',
                'dogId',
                'dogName',
                'weightInKg',
                'sex',
                'trainerId',
                'trainerState',
                'damId',
                'damName',
                'sireId',
                'sireName',
                'win',
                'dogAgeScaled',
                'lastFiveWinPercentage',
                'lastFivePlacePercentage',
                'weightInKgScaled',
                'rolling_box_win_percentage']
                 + feature_cols
                ]

feature_cols.extend(['dogAgeScaled',
                'lastFiveWinPercentage',
                'lastFivePlacePercentage',
                'weightInKgScaled',
                'rolling_box_win_percentage'])

#The below line will output your dataframe to a csv but may be too large to open in Excel.
#dataset.to_csv('testing.csv',index=False)

dataset shape: (1770293, 93)
dataset shape: (1770293, 93)
Processing dog rolling window 28D days
dataset shape: (1770293, 93)
Processing dog rolling window 91D days
dataset shape: (1770293, 93)
Processing dog rolling window 365D days
dataset shape: (1770293, 173)
Processing trainer rolling window 28D days
dataset shape: (1770293, 173)
Processing trainer rolling window 91D days
dataset shape: (1770293, 173)
Processing trainer rolling window 365D days
dataset shape: (1770293, 253)
Processing dam rolling window 28D days
dataset shape: (1770293, 253)
Processing dam rolling window 91D days
dataset shape: (1770293, 253)
Processing dam rolling window 365D days
dataset shape: (1770293, 333)
Processing sire rolling window 28D days
dataset shape: (1770293, 333)
Processing sire rolling window 91D days
dataset shape: (1770293, 333)
Processing sire rolling window 365D days


In [22]:
dataset.to_feather('topaz_data.fth')

In [23]:
dataset.shape

(1770293, 326)

In [66]:
for col in dataset.columns:
    print(col)

meetingDate
state
track
distance
raceId
raceTypeCode
raceNumber
boxNumber
rugNumber
runId
dogId
dogName
weightInKg
sex
trainerId
trainerState
damId
damName
sireId
sireName
win
dogAgeScaled
lastFiveWinPercentage
lastFivePlacePercentage
weightInKgScaled
rolling_box_win_percentage
dam_finishingPlaceMovement_max_28D
dam_finishingPlaceMovement_max_365D
dam_finishingPlaceMovement_max_91D
dam_finishingPlaceMovement_mean_28D
dam_finishingPlaceMovement_mean_365D
dam_finishingPlaceMovement_mean_91D
dam_finishingPlaceMovement_median_28D
dam_finishingPlaceMovement_median_365D
dam_finishingPlaceMovement_median_91D
dam_finishingPlaceMovement_min_28D
dam_finishingPlaceMovement_min_365D
dam_finishingPlaceMovement_min_91D
dam_finishingPlaceMovement_std_28D
dam_finishingPlaceMovement_std_365D
dam_finishingPlaceMovement_std_91D
dam_marginLog_max_28D
dam_marginLog_max_365D
dam_marginLog_max_91D
dam_marginLog_mean_28D
dam_marginLog_mean_365D
dam_marginLog_mean_91D
dam_marginLog_median_28D
dam_marginLog_med

In [65]:
feature_cols

['dam_finishingPlaceMovement_max_28D',
 'dam_finishingPlaceMovement_max_365D',
 'dam_finishingPlaceMovement_max_91D',
 'dam_finishingPlaceMovement_mean_28D',
 'dam_finishingPlaceMovement_mean_365D',
 'dam_finishingPlaceMovement_mean_91D',
 'dam_finishingPlaceMovement_median_28D',
 'dam_finishingPlaceMovement_median_365D',
 'dam_finishingPlaceMovement_median_91D',
 'dam_finishingPlaceMovement_min_28D',
 'dam_finishingPlaceMovement_min_365D',
 'dam_finishingPlaceMovement_min_91D',
 'dam_finishingPlaceMovement_std_28D',
 'dam_finishingPlaceMovement_std_365D',
 'dam_finishingPlaceMovement_std_91D',
 'dam_marginLog_max_28D',
 'dam_marginLog_max_365D',
 'dam_marginLog_max_91D',
 'dam_marginLog_mean_28D',
 'dam_marginLog_mean_365D',
 'dam_marginLog_mean_91D',
 'dam_marginLog_median_28D',
 'dam_marginLog_median_365D',
 'dam_marginLog_median_91D',
 'dam_marginLog_min_28D',
 'dam_marginLog_min_365D',
 'dam_marginLog_min_91D',
 'dam_marginLog_std_28D',
 'dam_marginLog_std_365D',
 'dam_marginLog_s

In [59]:
dataset.raceId.nunique()


84403

In [ ]:
# Your existing function to generate date range
def generate_date_range(start_date, end_date):
    start_date = start_date
    end_date = end_date

    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=31)

    return date_list

# Example usage:
start_date = datetime(2024,1,1)
end_date = (datetime.today() + timedelta(days=31))

# Generate the date range
date_range = generate_date_range(start_date, end_date)

In [5]:


def topaz_races_threaded(buckets, topaz_api, progress):
    all_races = []
    # print(f"{buckets=}")
    errors = []
    for bucket in buckets:
        start_date, end_date, state = bucket
        # print(bucket)
        try:
            races = topaz_api.get_races(from_date=start_date, to_date=end_date, owning_authority_code=state)
            all_races.append(races)
        except requests.HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            errors.append(bucket)
            pass
        # time.sleep(2)
        progress.update()
    return all_races,errors

def get_topaz_races(start_date, end_date, states, topaz_api:TopazAPI):
    date_range = generate_date_range(start_date, end_date)
    starts = date_range[:-1]
    ends = date_range[1:]
    date_range_states = [(start, end, state) for start, end in zip(starts, ends) for state in states]

    print(f"Created {len(date_range_states)} date ranges for {len(states)} states")

    num_workers = min(6, len(date_range_states))  # Adjust this value based on your system's capabilities
    chunk_size = math.ceil(len(date_range_states) / num_workers)

    chunks = [date_range_states[i:i + chunk_size] for i in range(0, len(date_range_states), chunk_size)]
    
    print(chunks)
    print(len(chunks))
    _process_jobs = []
    bars = []
    results = []
    errors = []
    for i in range(num_workers):
        bars.append(tqdm(total=len(chunks[i]), position=i)) 
        # time.sleep(2)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:


        for i,chunk in enumerate(chunks):
            _process_jobs.append(executor.submit(topaz_races_threaded, chunk, topaz_api, bars[i]))

        # results = []
        for job in concurrent.futures.as_completed(_process_jobs):
            result,error = job.result()
            errors.extend(error)
            results.append(result)

    

    # results = []
    print(errors)
    return results

In [8]:
start_date = datetime(2015,1,1)
end_date = (datetime.today() + timedelta(days=31))
states = states = ['NSW', 'VIC']
states = ['NZ']

In [9]:
output = get_topaz_races(start_date, end_date, states, topaz_api)
output_flat = [item for sublist in output for item in sublist]
all_races_df = pd.concat(output_flat,ignore_index=True).reset_index(drop=True)
all_races_df.to_csv('all_races_topas_NZ.csv',index=False)

Created 110 date ranges for 1 states
[[('2015-01-01', '2015-02-01', 'NZ'), ('2015-02-01', '2015-03-04', 'NZ'), ('2015-03-04', '2015-04-04', 'NZ'), ('2015-04-04', '2015-05-05', 'NZ'), ('2015-05-05', '2015-06-05', 'NZ'), ('2015-06-05', '2015-07-06', 'NZ'), ('2015-07-06', '2015-08-06', 'NZ'), ('2015-08-06', '2015-09-06', 'NZ'), ('2015-09-06', '2015-10-07', 'NZ'), ('2015-10-07', '2015-11-07', 'NZ'), ('2015-11-07', '2015-12-08', 'NZ'), ('2015-12-08', '2016-01-08', 'NZ'), ('2016-01-08', '2016-02-08', 'NZ'), ('2016-02-08', '2016-03-10', 'NZ'), ('2016-03-10', '2016-04-10', 'NZ'), ('2016-04-10', '2016-05-11', 'NZ'), ('2016-05-11', '2016-06-11', 'NZ'), ('2016-06-11', '2016-07-12', 'NZ'), ('2016-07-12', '2016-08-12', 'NZ')], [('2016-08-12', '2016-09-12', 'NZ'), ('2016-09-12', '2016-10-13', 'NZ'), ('2016-10-13', '2016-11-13', 'NZ'), ('2016-11-13', '2016-12-14', 'NZ'), ('2016-12-14', '2017-01-14', 'NZ'), ('2017-01-14', '2017-02-14', 'NZ'), ('2017-02-14', '2017-03-17', 'NZ'), ('2017-03-17', '2017-04

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

[]


In [12]:
all_races_df['date'] = pd.to_datetime(all_races_df['raceStart']).dt.date

In [15]:
meeting_ids = list(all_races_df['meetingId'].unique())

In [ ]:
def topaz_meeting_runs_threaded(chunk,topaz_api:TopazAPI,progress):
    race_runs = []
    race_results = []
    errors = []
    for race_id in chunk:
        try:
            # race_run = topaz_api.get_race_runs(race_id=race_id)
            # race_runs.append(race_run)
            time.sleep(0.3)
            race_result_json = topaz_api.get_race_result(race_id = race_id)
            try:
                race_run.to_feather(f"race_runs/{race_id}_run.fth")
                race_result_df = pd.DataFrame.from_dict([race_result_json])
                race_result_df.to_feather(f"results/{race_id}_results.fth")
            except Exception as e:
                print(e)

        except requests.HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            if http_err.response.status_code == 429:

                time.sleep(120)
            errors.append(race_id)
            pass
        progress.update()

    return race_runs,race_results,errors

def topaz_meeting_run_getter(race_id_list,topaz_api:TopazAPI):

    print(f"Fetching data for  {len(race_id_list)}")

    num_workers = 6
    chunk_size = math.ceil(len(race_id_list) / num_workers)

    chunks = [race_id_list[i:i + chunk_size] for i in range(0, len(race_id_list), chunk_size)]
    
    print(chunks)
    print(len(chunks))
    _process_jobs = []
    bars = []
    race_runs = []
    results = []
    errors = []
    for i in range(num_workers):
        bars.append(tqdm(total=len(chunks[i]), position=i)) 
        # time.sleep(2)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:


        for i,chunk in enumerate(chunks):
            _process_jobs.append(executor.submit(topaz_meeting_runs_threaded, chunk, topaz_api, bars[i]))

        # results = []
        for job in concurrent.futures.as_completed(_process_jobs):
            race_run,result_json,error = job.result()
            race_runs.extend(race_run)
            errors.extend(error)
            results.extend(result_json)

    

    # results = []
    print(errors)
    return race_runs,results,errors

In [16]:
race_ids = list(all_races_df['raceId'].unique())

In [2]:
race_id = 837931333

In [20]:
race_run = topaz_api.get_race_runs(race_id=race_id)
race_result_json,response = topaz_api.get_race_result(race_id=race_id)

In [21]:
rate_lim_left = int(response.headers['ratelimit-remaining'])
reset_time = int(response.headers['ratelimit-reset'])

In [22]:
reset_time

39

In [23]:
race_run_from_json = pd.DataFrame(race_result_json['runs'])

In [24]:
race_run_from_json 

,trackCode,track,distance,raceId,meetingDate,raceTypeCode,raceType,runId,dogId,dogName,...,sireName,dateWhelped,totalFormCount,last5,isLateScratching,bestTime,bestFinishTrackAndDistance,pir,careerPrizeMoney,averageSpeed
0,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631038,668114567,MAGIC MUFFIN,...,FERNANDO BALE,2021-01-19T00:00:00.000Z,0,3,False,NBT,29.676,None,131695.0,66.616000
1,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631035,662312481,SCOOBY BOO,...,BERNARDO,2021-01-01T00:00:00.000Z,0,1,False,30.69,30.397,2222,6080.0,65.646667
2,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631041,678122554,GABRIELLA MAREE,...,FERNANDO BALE,2021-03-08T00:00:00.000Z,0,4-3,False,NBT,30.592,3347,9595.0,64.428000
3,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631036,644458849,WINSTON TOWN,...,BERNARDO,2020-11-10T00:00:00.000Z,0,2-4-2-7-2,False,NBT,30.137,4458,56570.0,66.891000
4,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631037,637973470,SERENA BALE,...,ORSON ALLEN,2020-10-16T00:00:00.000Z,0,2-6-2-2-6,False,NBT,29.925,8533,58380.0,65.857000
5,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631040,669309455,TRENDING XARA,...,BERNARDO,2021-01-31T00:00:00.000Z,0,4-7-4-2-6,False,NBT,30.155,7664,77095.0,66.274000
6,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631039,668114568,NEVER BE ALONE,...,FERNANDO BALE,2021-01-19T00:00:00.000Z,0,3,False,NBT,30.071,1111,55800.0,66.659000
7,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631034,643584615,HURRICANE HAZEL,...,COSMIC RUMBLE,2020-12-05T00:00:00.000Z,0,1-7-4-2-7,False,30.66,30.663,6886,8545.0,66.369000
8,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631043,668114566,MAKE NO PROMISE,...,FERNANDO BALE,2021-01-19T00:00:00.000Z,0,5,False,NBT,29.667,None,197640.0,65.852000
9,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631042,677833587,MY TIME,...,BARCIA BALE,2021-02-25T00:00:00.000Z,0,5-7,False,NBT,31.002,5775,56520.5,61.220000


In [36]:
split_times = pd.DataFrame(race_result_json['splitTimes'])
split_times_1 = split_times[split_times['splitTimeMarker'] == 1][['runId','time','position','splitMargin']]
split_time_2 = split_times[split_times['splitTimeMarker'] == 2][['runId','time','position','splitMargin']]
split_times = split_times_1.merge(split_time_2, on='runId',suffixes=('_1','_2'),how='left')

In [37]:
split_times

,runId,time_1,position_1,splitMargin_1,time_2,position_2,splitMargin_2
0,840631039,5.05000,1,NaN,17.78,1.0,NaN
1,840631035,5.20750,2,0.15750,NaN,NaN,NaN
2,840631041,5.20925,3,0.00175,NaN,NaN,NaN
3,840631036,5.21100,4,0.00175,NaN,NaN,NaN
4,840631042,5.26350,5,0.05250,NaN,NaN,NaN
5,840631034,5.29850,6,0.03500,NaN,NaN,NaN
6,840631040,5.30025,7,0.00175,NaN,NaN,NaN
7,840631037,5.33525,8,0.03500,NaN,NaN,NaN


In [42]:
race_run_from_json 

,trackCode,track,distance,raceId,meetingDate,raceTypeCode,raceType,runId,dogId,dogName,...,sireName,dateWhelped,totalFormCount,last5,isLateScratching,bestTime,bestFinishTrackAndDistance,pir,careerPrizeMoney,averageSpeed
0,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631038,668114567,MAGIC MUFFIN,...,FERNANDO BALE,2021-01-19T00:00:00.000Z,0,3,False,NBT,29.676,None,131695.0,66.616000
1,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631035,662312481,SCOOBY BOO,...,BERNARDO,2021-01-01T00:00:00.000Z,0,1,False,30.69,30.397,2222,6080.0,65.646667
2,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631041,678122554,GABRIELLA MAREE,...,FERNANDO BALE,2021-03-08T00:00:00.000Z,0,4-3,False,NBT,30.592,3347,9595.0,64.428000
3,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631036,644458849,WINSTON TOWN,...,BERNARDO,2020-11-10T00:00:00.000Z,0,2-4-2-7-2,False,NBT,30.137,4458,56570.0,66.891000
4,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631037,637973470,SERENA BALE,...,ORSON ALLEN,2020-10-16T00:00:00.000Z,0,2-6-2-2-6,False,NBT,29.925,8533,58380.0,65.857000
5,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631040,669309455,TRENDING XARA,...,BERNARDO,2021-01-31T00:00:00.000Z,0,4-7-4-2-6,False,NBT,30.155,7664,77095.0,66.274000
6,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631039,668114568,NEVER BE ALONE,...,FERNANDO BALE,2021-01-19T00:00:00.000Z,0,3,False,NBT,30.071,1111,55800.0,66.659000
7,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631034,643584615,HURRICANE HAZEL,...,COSMIC RUMBLE,2020-12-05T00:00:00.000Z,0,1-7-4-2-7,False,30.66,30.663,6886,8545.0,66.369000
8,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631043,668114566,MAKE NO PROMISE,...,FERNANDO BALE,2021-01-19T00:00:00.000Z,0,5,False,NBT,29.667,None,197640.0,65.852000
9,MEA,The Meadows,525,837931333,2022-11-11T00:00:00.000Z,MF,Maiden Final,840631042,677833587,MY TIME,...,BARCIA BALE,2021-02-25T00:00:00.000Z,0,5-7,False,NBT,31.002,5775,56520.5,61.220000


In [40]:
race_result_json

{'raceId': 837931333,
 'meetingId': 809592457,
 'raceNumber': 1,
 'raceTypeCode': 'MF',
 'raceType': 'Maiden Final',
 'name': 'SPORTSBET SET A DEPOSIT LIMIT FINAL',
 'startTime': '7:12PM',
 'raceStart': '2022-11-11T08:12:00.000Z',
 'distance': 525,
 'isBoxDrawn': True,
 'prizeMoney1': 5900,
 'prizeMoney2': 1820,
 'prizeMoney3': 915,
 'prizeMoney4': 455,
 'prizeMoney5': 0,
 'prizeMoney6': 0,
 'prizeMoney7': 0,
 'prizeMoney8': 0,
 'prizeMoneyTotal': 9090,
 'analystComment': 'SCOOBY BOO (1) and WINSTON TOWN (3) had a great battle in their heat, with the former leading all the way and just doing enough. NEVER BE ALONE (6) opened up a huge lead in the other heat before being run down by HURRICANE HAZEL (7).',
 'suggestedBetTypeCode': 'Win',
 'suggestedBet': '1',
 'isDailyDouble': False,
 'isQuaddie': False,
 'isEarlyQuaddie': True,
 'isRunningDouble': True,
 'isTrifecta': True,
 'isQuinella': True,
 'isExacta': True,
 'isPickFour': True,
 'isBixSix': False,
 'tip1': 1,
 'tip2': 3,
 'tip3': 

In [55]:
race_result_json_meeting = topaz_api.get_meeting_details(meeting_id = 809592457)

HTTPError: 401 Client Error: Unauthorized for url: https://topaz.grv.org.au/api/meeting/809592457?format=all

In [39]:
race_result_df

,raceId,meetingId,raceNumber,raceTypeCode,raceType,name,startTime,raceStart,distance,isBoxDrawn,...,isRaceResultEntered,photoFinishUrl,resultYouTubeVideoId,tabTipRunners,runs,dividends,exoticBetType,positionInRunning,splitTimes,resultsSummary
0,926131298,926130584,3,5,Grade 5,LADBROKES EASY FORM 1 WIN,7:17PM,2023-08-04T09:17:00.000Z,400,False,...,True,None,None,"[None, None, None, None]","[{'trackCode': 'WAG', 'track': 'Wagga', 'dista...",[],[],[],[],"{'first': 2, 'second': 6, 'third': 3, 'fourth'..."


In [3]:
def topaz_race_runs_threaded(chunk,topaz_api:TopazAPI,progress):
    race_runs = []
    race_results = []
    errors = []
    for race_id in chunk:
        try:
            race_run = topaz_api.get_race_runs(race_id=race_id)
            race_runs.append(race_run)
            time.sleep(0.3)
            race_result_json = topaz_api.get_race_result(race_id = race_id)
            try:
                race_run.to_feather(f"race_runs/{race_id}_run.fth")
                race_result_df = pd.DataFrame.from_dict([race_result_json])
                race_result_df.to_feather(f"results/{race_id}_results.fth")
            except Exception as e:
                print(e)

        except requests.HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
            if http_err.response.status_code == 429:

                time.sleep(120)
            errors.append(race_id)
            pass
        progress.update()

    return race_runs,race_results,errors

In [4]:
def topaz_race_run_getter(race_id_list,topaz_api:TopazAPI):

    print(f"Fetching data for  {len(race_id_list)}")

    num_workers = 6
    chunk_size = math.ceil(len(race_id_list) / num_workers)

    chunks = [race_id_list[i:i + chunk_size] for i in range(0, len(race_id_list), chunk_size)]
    
    print(chunks)
    print(len(chunks))
    _process_jobs = []
    bars = []
    race_runs = []
    results = []
    errors = []
    for i in range(num_workers):
        bars.append(tqdm(total=len(chunks[i]), position=i)) 
        # time.sleep(2)
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:


        for i,chunk in enumerate(chunks):
            _process_jobs.append(executor.submit(topaz_race_runs_threaded, chunk, topaz_api, bars[i]))

        # results = []
        for job in concurrent.futures.as_completed(_process_jobs):
            race_run,result_json,error = job.result()
            race_runs.extend(race_run)
            errors.extend(error)
            results.extend(result_json)

    

    # results = []
    print(errors)
    return race_runs,results,errors

In [3]:
all_races_df = pd.read_csv('all_races_topas.csv', header=0)
i = 0
race_ids = list(all_races_df['raceId'].unique())
# subset_ids = race_ids[i:min(len(race_ids),i+100)]
# race_runs,results,errors = topaz_race_run_getter(subset_ids,topaz_api)

In [4]:
all_races_df

,raceId,meetingId,raceNumber,raceTypeCode,raceType,name,startTime,raceStart,distance,isBoxDrawn,...,tip3,tip4,winPool,placePool,trifectaPool,owningAuthorityCode,isRaceResultEntered,photoFinishUrl,resultYouTubeVideoId,tabTipRunners
0,841164103,841163481.0,1,M,Maiden,WELCOME TO GUNNEDAH MAIDEN,3:37PM,2022-11-10T04:37:00.000Z,340,False,...,NaN,NaN,NaN,NaN,NaN,NSW,True,NaN,NaN,"[None, None, None, None]"
1,841164104,841163481.0,2,M,Maiden,FREE ENTRY AT GUNNEDAH MAIDEN,3:52PM,2022-11-10T04:52:00.000Z,340,False,...,NaN,NaN,NaN,NaN,NaN,NSW,True,NaN,NaN,"[None, None, None, None]"
2,841164105,841163481.0,3,5,Grade 5,LADBROKES ODDS BOOST,4:12PM,2022-11-10T05:12:00.000Z,431,False,...,NaN,NaN,NaN,NaN,NaN,NSW,True,NaN,NaN,"[None, None, None, None]"
3,841164106,841163481.0,4,X45,Mixed 4/5,LADBROKE IT!,4:39PM,2022-11-10T05:39:00.000Z,340,False,...,NaN,NaN,NaN,NaN,NaN,NSW,True,NaN,NaN,"[None, None, None, None]"
4,841164107,841163481.0,5,NG,Non Graded,LADBROKES MATES MODE,4:54PM,2022-11-10T05:54:00.000Z,340,False,...,NaN,NaN,NaN,NaN,NaN,NSW,True,NaN,NaN,"[None, None, None, None]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280248,475655924,463329830.0,8,X45,Mixed 4/5,SALE MOTOR GROUP,1:49PM,2019-09-19T03:49:00.000Z,520,True,...,7.0,1.0,8811.22,3282.65,10219.80,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,76RdirUgLhE,"[5, 4, 7, 1]"
280249,475655914,463329830.0,9,X67H,Mixed 6/7 Heat,YES OPTUS HT2,2:09PM,2019-09-19T04:09:00.000Z,440,True,...,6.0,7.0,7080.29,1891.69,6827.93,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,hbA-4cR7TE8,"[2, 8, 6, 7]"
280250,475655915,463329830.0,10,X67H,Mixed 6/7 Heat,ROTTI SECURITY HT3,2:24PM,2019-09-19T04:24:00.000Z,440,True,...,5.0,4.0,8563.97,2564.35,7474.56,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,RFZzWNzrtjM,"[3, 1, 5, 4]"
280251,475655916,463329830.0,11,X67H,Mixed 6/7 Heat,DUNKLEY'S MACHINERY HT4,2:44PM,2019-09-19T04:44:00.000Z,440,True,...,3.0,1.0,9132.25,2245.90,7388.67,VIC,True,https://fasttrack.blob.core.windows.net/webcon...,V25YIm6oxMs,"[7, 8, 3, 1]"


In [44]:
test_race_run_df = pd.read_feather('race_runs/837931333_run.fth')

In [30]:
test_race_results_json_df = pd.read_feather('results/837931333_results.fth')

In [31]:
test_race_results_json_df

,raceId,meetingId,raceNumber,raceTypeCode,raceType,name,startTime,raceStart,distance,isBoxDrawn,...,isRaceResultEntered,photoFinishUrl,resultYouTubeVideoId,tabTipRunners,runs,dividends,exoticBetType,positionInRunning,splitTimes,resultsSummary
0,837931333,809592457,1,MF,Maiden Final,SPORTSBET SET A DEPOSIT LIMIT FINAL,7:12PM,2022-11-11T08:12:00.000Z,525,True,...,True,https://fasttrack.blob.core.windows.net/webcon...,Or36a0H08_A,"[1, 3, 6, 7]","[{'averageSpeed': 66.616, 'bestFinishTrackAndD...","[{'dividend': 5.2, 'dividendType': 'Win', 'dog...","[{'dividend': 51.6, 'exoticTypeCode': 'Exacta'...","[{'boxNumber': 6, 'dogId': 668114568, 'positio...","[{'dogId': 668114568, 'dogName': 'NEVER BE ALO...","{'first': 4, 'fourth': 5, 'second': 6, 'third'..."


In [7]:
for i in range(0,len(race_ids),100):
    subset_ids = race_ids[i:min(len(race_ids),i+1000)]
    race_runs,results,errors = topaz_race_run_getter(subset_ids,topaz_api)

    results_df = pd.DataFrame.from_dict(results)
    all_race_runs = pd.concat(race_runs,ignore_index=True).reset_index(drop=True)
    all_race_runs.to_feather(f'race_runs/{i}_topaz_race_runs.fth')
    results_df.to_feather(f"results/{i}_topaz_results.fth")
    # with 

Fetching data for  1000
[[841164103, 841164104, 841164105, 841164106, 841164107, 841164108, 841164109, 841164110, 841164111, 841164112, 841164113, 841186562, 841186563, 841186564, 841186565, 841186566, 841186567, 841186568, 841186569, 841186570, 841186571, 841186572, 842794260, 842794261, 842794262, 842794263, 842794264, 842794265, 842794266, 842794267, 842794268, 842794269, 842794270, 842794271, 842794284, 842794285, 842794286, 842794287, 842794288, 842794289, 842794290, 842794291, 842794292, 842794293, 842794294, 842794295, 842799748, 842799749, 842799750, 842799752, 842799754, 842799756, 842799758, 842799760, 842799763, 842799765, 842799767, 842799768, 842799751, 842799753, 842799755, 842799757, 842799759, 842799761, 842799762, 842799764, 842799766, 842799769, 842799770, 842799783, 842799784, 842799785, 842799786, 842799787, 842799788, 842799789, 842799790, 842799791, 842799792, 842799793, 842794308, 842794309, 842794310, 842794311, 842794312, 842794313, 842794314, 842794315, 842794

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

...

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/165 [00:00<?, ?it/s]

In [14]:
results

[{'raceId': 842794313,
  'meetingId': 842793730,
  'raceNumber': 6,
  'raceTypeCode': '5',
  'raceType': 'Grade 5',
  'name': 'GDD CHARITY CHALLENGE 2',
  'startTime': '8:12PM',
  'raceStart': '2022-11-12T09:12:00.000Z',
  'distance': 520,
  'isBoxDrawn': False,
  'prizeMoney1': 6000,
  'prizeMoney2': 1700,
  'prizeMoney3': 1200,
  'prizeMoney4': 100,
  'prizeMoney5': 0,
  'prizeMoney6': 0,
  'prizeMoney7': 0,
  'prizeMoney8': 0,
  'prizeMoneyTotal': 9000,
  'analystComment': None,
  'suggestedBetTypeCode': None,
  'suggestedBet': None,
  'isDailyDouble': False,
  'isQuaddie': False,
  'isEarlyQuaddie': False,
  'isRunningDouble': False,
  'isTrifecta': False,
  'isQuinella': False,
  'isExacta': False,
  'isPickFour': False,
  'isBixSix': False,
  'tip1': None,
  'tip2': None,
  'tip3': None,
  'tip4': None,
  'winPool': None,
  'placePool': None,
  'trifectaPool': None,
  'owningAuthorityCode': 'NSW',
  'isRaceResultEntered': True,
  'photoFinishUrl': None,
  'resultYouTubeVideoId': 

In [10]:
race_ids = list(all_races_df['raceId'].unique())
code = "NSW-VIC"
for race_id in tqdm(race_ids, desc="Processing races", unit="race"):
    result_retries = 10

    while result_retries > 0:
        # Use tqdm to create a progress bar
        # Get race run data
        try:
            race_run = topaz_api.get_race_runs(race_id=race_id)
            race_result_json = topaz_api.get_race_result(race_id=race_id)
            file_path = code + '_DATA.csv'
            file_exists = os.path.isfile(file_path)
            header_param = not file_exists

            race_result = pd.DataFrame.from_dict([race_result_json])
            split_times_df = pd.DataFrame(race_result['splitTimes'].tolist(),index=race_result.index)

            splits_dict = split_times_df.T.stack().to_frame()
            splits_dict.reset_index(drop=True, inplace= True)
            splits_normalised = pd.json_normalize(splits_dict[0])

            if len(splits_normalised) == 0:
                race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                break

            first_split = splits_normalised[splits_normalised['splitTimeMarker'] == 1]
            first_split = first_split[['runId','position','time']]
            first_split = first_split.rename(columns={'position':'firstSplitPosition','time':'firstSplitTime'})
            second_split = splits_normalised[splits_normalised['splitTimeMarker'] == 2]
            second_split = second_split[['runId','position','time']]
            second_split = second_split.rename(columns={'position':'secondSplitPosition','time':'secondSplitTime'})

            split_times = splits_normalised[['runId']]
            split_times = pd.merge(split_times,first_split,how='left',on=['runId'])
            split_times = pd.merge(split_times,second_split,how='left',on=['runId'])

            race_run = pd.merge(race_run,split_times,how='left',on=['runId'])
            race_run.drop_duplicates(inplace=True)
            race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
            break
        except requests.HTTPError as http_err:
            if http_err.response.status_code == 404:
                file_path = code + '_DATA.csv'
                file_exists = os.path.isfile(file_path)
                header_param = not file_exists
                race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                break
        except Exception as e:
            print(race_id)
            result_retries -= 1
            if result_retries > 0:
                time.sleep(15)
            else:
                time.sleep(120)

Processing races:   0%|          | 0/260964 [00:00<?, ?race/s]

KeyboardInterrupt: 

In [ ]:
# Iterate over 7-day blocks
for i in range(0, len(date_range), 10):
    start_block_date = date_range[i]
    print(start_block_date)
    end_block_date = date_range[min(i + 9, len(date_range) - 1)]  # Ensure the end date is within the range

    codes = ['NT','VIC','NSW','SA','WA','QLD','TAS','NZ']
    codes = ['VIC', "NSW"]
    all_races = []
    for code in codes:
        
        print(code)
        retries = 10  # Number of retries
        while retries > 0:
            try:
                races = topaz_api.get_races(from_date=start_block_date, to_date=end_block_date, owning_authority_code=code)
                all_races.append(races)
                break  # Break out of the loop if successful
            except requests.HTTPError as http_err:
                if http_err.response.status_code == 429:
                    retries -= 1
                    if retries > 0:
                        print(f"Rate limited. Retrying in 121 seconds...")
                        time.sleep(121)
                    else:
                        print("Max retries reached. Moving to the next block.")
                else:
                    print(f"Error fetching races for {code}: {http_err.response.status_code}")
                    retries -= 1
                    if retries > 0:
                        print(f"Retrying in 30 seconds...")
                        time.sleep(30)
                    else:
                        print("Max retries reached. Moving to the next block.")

    try:
        all_races_df = pd.concat(all_races,ignore_index=True).reset_index(drop=True)
    except ValueError:
        continue

    # Extract unique race IDs
    race_ids = list(all_races_df['raceId'].unique())

    for race_id in tqdm(race_ids, desc="Processing races", unit="race"):
        result_retries = 10

        while result_retries > 0:
            # Use tqdm to create a progress bar
            # Get race run data
            try:
                race_run = topaz_api.get_race_runs(race_id=race_id)
                race_result_json = topaz_api.get_race_result(race_id=race_id)
                file_path = code + '_DATA.csv'
                file_exists = os.path.isfile(file_path)
                header_param = not file_exists

                race_result = pd.DataFrame.from_dict([race_result_json])
                split_times_df = pd.DataFrame(race_result['splitTimes'].tolist(),index=race_result.index)

                splits_dict = split_times_df.T.stack().to_frame()
                splits_dict.reset_index(drop=True, inplace= True)
                splits_normalised = pd.json_normalize(splits_dict[0])

                if len(splits_normalised) == 0:
                    race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                    break

                first_split = splits_normalised[splits_normalised['splitTimeMarker'] == 1]
                first_split = first_split[['runId','position','time']]
                first_split = first_split.rename(columns={'position':'firstSplitPosition','time':'firstSplitTime'})
                second_split = splits_normalised[splits_normalised['splitTimeMarker'] == 2]
                second_split = second_split[['runId','position','time']]
                second_split = second_split.rename(columns={'position':'secondSplitPosition','time':'secondSplitTime'})

                split_times = splits_normalised[['runId']]
                split_times = pd.merge(split_times,first_split,how='left',on=['runId'])
                split_times = pd.merge(split_times,second_split,how='left',on=['runId'])

                race_run = pd.merge(race_run,split_times,how='left',on=['runId'])
                race_run.drop_duplicates(inplace=True)
                race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                break
            except requests.HTTPError as http_err:
                if http_err.response.status_code == 404:
                    file_path = code + '_DATA.csv'
                    file_exists = os.path.isfile(file_path)
                    header_param = not file_exists
                    race_run.to_csv(code + '_DATA.csv', mode='a', header=header_param, index=False)
                    break
            except Exception as e:
                print(race_id)
                result_retries -= 1
                if result_retries > 0:
                    time.sleep(15)
                else:
                    time.sleep(120)